In [14]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.0+cu117


'%.4f'

In [15]:
lib.utils.set_seed()
downscale = (14, 14)
test_x, test_y = lib.datasets.to_plain(*lib.datasets.get_mnist_test(), downscale=downscale, to_odd=True)
train_x, train_y = lib.datasets.to_plain(*lib.datasets.get_mnist_train(), downscale=downscale, to_odd=True)
network = lib.livenet.LiveNet.create_perceptron(train_x.shape[1], None, 2)

In [8]:
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = lib.nets.criterion_n
optimizer = lib.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=100)


LiveNet


In [9]:
network.context.alpha_l1 = 0.01
trainer.step(25000)


Iˈ0.000 1.018 = 1.005+0.013 lib/trainer.py:92
Iˈ0.158 0.963 = 0.939+0.024 lib/trainer.py:92
Iˈ0.301 0.915 = 0.858+0.056 lib/trainer.py:92
Iˈ0.314 killing S1->D0 at tick 210 with k=0.009 lib/livenet.py:191
Iˈ0.336 killing S0->D1 at tick 229 with k=0.007 lib/livenet.py:191
Iˈ0.412 0.899 = 0.817+0.082 lib/trainer.py:92
Iˈ0.518 0.906 = 0.801+0.104 lib/trainer.py:92
Iˈ0.627 0.924 = 0.787+0.136 lib/trainer.py:92
Iˈ0.662 killing S0->D0 at tick 535 with k=0.005 lib/livenet.py:191
Iˈ0.662 S0 became useless at tick 535 lib/livenet.py:61
Iˈ0.662 There is no death for S0 lib/livenet.py:44
Iˈ0.724 0.931 = 0.774+0.157 lib/trainer.py:92
Iˈ0.818 0.937 = 0.767+0.170 lib/trainer.py:92
Iˈ0.916 0.915 = 0.747+0.168 lib/trainer.py:92
Iˈ1.015 0.912 = 0.748+0.164 lib/trainer.py:92
Iˈ1.110 0.931 = 0.775+0.156 lib/trainer.py:92
Iˈ1.203 0.919 = 0.778+0.141 lib/trainer.py:92
Iˈ1.303 0.904 = 0.756+0.148 lib/trainer.py:92
Iˈ1.395 0.909 = 0.755+0.154 lib/trainer.py:92
Iˈ1.489 0.902 = 0.770+0.131 lib/trainer.py:92
Iˈ

In [10]:
pred = network(test_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = test_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.2653

In [23]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["S262->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_amount)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
# accum.plot()

Using matplotlib backend: TkAgg


In [11]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000.):
    print(name, f"{p.item():.1f}")

D0 2.6
D1 -2.7
S1->D1 1.6
S2->D1 -7.1
S3->D1 -1.5


In [19]:
network._parameters

OrderedDict([('D0',
              Parameter containing:
              tensor(-0.0146, requires_grad=True)),
             ('D1',
              Parameter containing:
              tensor(0.0146, requires_grad=True)),
             ('N4',
              Parameter containing:
              tensor(-0.0045, requires_grad=True))])

In [151]:
lib.livenet.export_onnx(network, "/home/spometun/model.onnx")

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

